
## Notebook Summary

This notebook covers the following steps for preparing labeled training data for product recommendation:

1. **Set Global Cutoff Timestamp:**  
   Defined a cutoff time (`CUTOFF_TS`) to separate training features from future purchases.

2. **Load Feature Data:**  
   Loaded leakage-free candidate features from the feature store table.

3. **Extract Future Purchases:**  
   Selected purchases strictly after the cutoff timestamp to use as positive labels.

4. **Attach Labels:**  
   Joined features with future purchases, assigning label `1` for future purchases and `0` otherwise.

5. **Sanity Check:**  
   Displayed label distribution to verify correct labeling.

6. **Save Labeled Data:**  
   Saved the final labeled dataset as a Delta table for downstream model training.

In [0]:
from pyspark.sql import functions as F

gold = "kusha_solutions.product_recomendation"

# ============================================================
# 🔒 GLOBAL CUTOFF (MUST MATCH FEATURE ENGINEERING)
# ============================================================
CUTOFF_TS = "2025-12-01 00:00:00"

print("📌 Label cutoff time:", CUTOFF_TS)

# ============================================================
# 1️⃣ READ FINAL FEATURE DATA (LEAKAGE-FREE)
# ============================================================
features = spark.table(
    "kusha_solutions.product_recomendation.fs_canddiate_features"
)

print("✅ Features loaded")

# ============================================================
# 2️⃣ FUTURE PURCHASES (LABEL SOURCE)
#     → STRICTLY AFTER CUTOFF
# ============================================================
future_purchases = (
    spark.table(f"{gold}.gold_sales_enriched")
         .filter(F.col("EventTime") >= F.lit(CUTOFF_TS))
         .filter(F.lower(F.col("InteractionType")) == "purchase")
         .select("CustomerID", "ProductID")
         .distinct()
         .withColumn("label", F.lit(1))
)

print("✅ Future purchases extracted")

# ============================================================
# 3️⃣ LABEL ATTACHMENT
# ============================================================
labeled_data = (
    features
      .join(
          future_purchases,
          ["CustomerID", "ProductID"],
          "left"
      )
      .withColumn("label", F.coalesce(F.col("label"), F.lit(0)))
)

# ============================================================
# 4️⃣ SANITY CHECK
# ============================================================
print("📊 Label distribution:")
labeled_data.groupBy("label").count().show()

# ============================================================
# 5️⃣ SAVE LABELED DATA
# ============================================================
labeled_data.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable(f"{gold}.training_labeled_candidates_v2")

print("✅ Labeled data saved successfully")
